In [1]:
import os
import pathlib
import json

In [2]:
import pandas as pd 
import numpy as np

### enter bulletin directory 

In [3]:
bulletin_dir = 'FMA_2020'

In [4]:
dpath = pathlib.Path.cwd().parents[1] / 'bulletin' / bulletin_dir

In [5]:
dpath

PosixPath('/home/nicolasf/operational/ICU/ops/bulletin/FMA_2020')

### reads the country dictionnary for the TRMM rainfall time-series

In [6]:
with open(pathlib.Path.cwd().parents[1] / 'resources' / 'dict_countries.json', 'r') as fj: 
    dict_countries = json.load(fj)

### read the forecast table 

In [157]:
forecast = pd.read_csv(dpath / 'copernicus_forecasts_{}.csv'.format(bulletin_dir), index_col=0)

In [158]:
forecast.head()

,T1,T2,T3,country_fname,std,CONFIDENCE
country_name,,,,,,
Marshall Islands,3,17,80,Marshall_Islands,16.244933,Moderate-High
Federated States of Micronesia,20,22,58,Federated_States_of_Micronesia,20.239218,Moderate
Kiribati: Line Islands,21,33,46,Kiribati:_Line_Islands,11.440509,High
Papua New Guinea,25,30,45,Papua_New_Guinea,9.535794,High
Nauru,29,33,38,Nauru,28.115259,Moderate


In [159]:
def terciles_to_category(df, colnames=['T1','T2','T3'], mindiff=10, climdiff=5): 
    """
    convert a tercile probabilistic forecast into a categorical forecast
    
    if the difference between the maximum probability and the next one is >= mindiff 
    (10 is default) then it's either 'ABOVE', 'NEAR NORMAL' or 'BELOW' depending on the max 
    probability 
    
    if the difference between consecutive probabilities is less than mindiff (10)
    but greater than climdiff (5) then we end up with either 'AVG - BELOW' or 'AVG - ABOVE'
    
    if the difference between consecutive probabilities is less than 5%, it's a CLIMATOLOGY 
    forecasts 
    """
    
    T1, T2, T3 = colnames
    
    cats = []
    for i, row in df.iterrows(): 
        fcst = row.loc[colnames,].astype('int')
        if (fcst.idxmax()  == T3) & ((fcst.loc[T3] - fcst.loc[T2]) >= mindiff): 
            cat = 'ABOVE'
#         elif (fcst.idxmax()  == T3) & ((fcst.loc[T3] - fcst.loc[T2]) < mindiff) & ((fcst.loc[T3] - fcst.loc[T2]) > climdiff) & ((fcst.loc[T3] - fcst.loc[T1]) > mindiff):
        elif (fcst.idxmax()  == T3) & ((fcst.loc[T3] - fcst.loc[T2]) < mindiff) & ((fcst.loc[T3] - fcst.loc[T1]) > mindiff):
            cat = 'AVG - ABOVE'
        elif (fcst.idxmax()  == T1) & ((fcst.loc[T1] - fcst.loc[T2]) >= mindiff):
            cat = 'BELOW'
#         elif (fcst.idxmax()  == T1) & ((fcst.loc[T1] - fcst.loc[T2]) < mindiff) & ((fcst.loc[T1] - fcst.loc[T2]) > climdiff) & ((fcst.loc[T1] - fcst.loc[T3]) > mindiff):
        elif (fcst.idxmax()  == T1) & ((fcst.loc[T1] - fcst.loc[T2]) < mindiff) & ((fcst.loc[T1] - fcst.loc[T3]) > mindiff):
            cat = 'AVG - BELOW'
        elif (fcst.idxmax()  == T2) & ((fcst.loc[T2] - fcst.loc[T1]) >= mindiff) & ((fcst.loc[T2] - fcst.loc[T3]) >= mindiff): 
            cat = 'NEAR NORMAL'
        else: 
            cat = 'CLIMATOLOGY'
        cats.append(cat)
    return np.array(cats)

In [160]:
forecast

,T1,T2,T3,country_fname,std,CONFIDENCE
country_name,,,,,,
Marshall Islands,3,17,80,Marshall_Islands,16.244933,Moderate-High
Federated States of Micronesia,20,22,58,Federated_States_of_Micronesia,20.239218,Moderate
Kiribati: Line Islands,21,33,46,Kiribati:_Line_Islands,11.440509,High
Papua New Guinea,25,30,45,Papua_New_Guinea,9.535794,High
Nauru,29,33,38,Nauru,28.115259,Moderate
Vanuatu North,32,34,34,Vanuatu_North,19.573168,Moderate-High
Palau,32,35,33,Palau,30.241486,Moderate
Tonga,32,36,32,Tonga,17.671547,Moderate-High
Niue,34,34,32,Niue,21.046265,Moderate


In [161]:
forecast.loc[:,'CATEGORY'] = terciles_to_category(forecast)

In [162]:
forecast

,T1,T2,T3,country_fname,std,CONFIDENCE,CATEGORY
country_name,,,,,,,
Marshall Islands,3,17,80,Marshall_Islands,16.244933,Moderate-High,ABOVE
Federated States of Micronesia,20,22,58,Federated_States_of_Micronesia,20.239218,Moderate,ABOVE
Kiribati: Line Islands,21,33,46,Kiribati:_Line_Islands,11.440509,High,ABOVE
Papua New Guinea,25,30,45,Papua_New_Guinea,9.535794,High,ABOVE
Nauru,29,33,38,Nauru,28.115259,Moderate,CLIMATOLOGY
Vanuatu North,32,34,34,Vanuatu_North,19.573168,Moderate-High,CLIMATOLOGY
Palau,32,35,33,Palau,30.241486,Moderate,CLIMATOLOGY
Tonga,32,36,32,Tonga,17.671547,Moderate-High,CLIMATOLOGY
Niue,34,34,32,Niue,21.046265,Moderate,CLIMATOLOGY


### IMPORTANT, IF THE ATTRIBUTED CATEGORIES DO NOT MAKE SENSE, PLEASE CHANGE THEM 'MANUALLY'

if needed you can modify the category given for the forecast like this: 
        
```Python
forecast.loc['Marquesas','CATEGORY'] = 'AVG-BELOW'
```
----- 

In [163]:
forecast.loc['Nauru','CATEGORY'] = 'AVG-ABOVE'

In [164]:
forecast.loc['Fiji','CATEGORY'] = 'NEAR NORMAL'

In [165]:
forecast.loc['Austral Islands','CATEGORY'] = 'AVG-BELOW'

In [166]:
forecast

,T1,T2,T3,country_fname,std,CONFIDENCE,CATEGORY
country_name,,,,,,,
Marshall Islands,3,17,80,Marshall_Islands,16.244933,Moderate-High,ABOVE
Federated States of Micronesia,20,22,58,Federated_States_of_Micronesia,20.239218,Moderate,ABOVE
Kiribati: Line Islands,21,33,46,Kiribati:_Line_Islands,11.440509,High,ABOVE
Papua New Guinea,25,30,45,Papua_New_Guinea,9.535794,High,ABOVE
Nauru,29,33,38,Nauru,28.115259,Moderate,AVG-ABOVE
Vanuatu North,32,34,34,Vanuatu_North,19.573168,Moderate-High,CLIMATOLOGY
Palau,32,35,33,Palau,30.241486,Moderate,CLIMATOLOGY
Tonga,32,36,32,Tonga,17.671547,Moderate-High,CLIMATOLOGY
Niue,34,34,32,Niue,21.046265,Moderate,CLIMATOLOGY


### saves to the folder for inclusion in the PPT 

#### This is the table that appears on page 2 of the ICU bulletin

In [167]:
ppt_forecast_table = forecast.loc[:,['T1','T2','T3','CATEGORY','CONFIDENCE']]

In [168]:
ppt_forecast_table = ppt_forecast_table.sort_values(by=['T3','T2','T1'], ascending=False)

In [169]:
ppt_forecast_table = ppt_forecast_table.rename(index={'Tuamotu / Gambier Islands':'Tuamotu Islands'})

In [170]:
ppt_forecast_table = ppt_forecast_table.rename(index={'Federated States of Micronesia':'FSM'})

In [171]:
ppt_forecast_table.to_csv(dpath / 'ICU_forecast_table_for_PPT_{}.csv'.format(bulletin_dir))

In [172]:
ppt_forecast_table

,T1,T2,T3,CATEGORY,CONFIDENCE
country_name,,,,,
Marshall Islands,3,17,80,ABOVE,Moderate-High
FSM,20,22,58,ABOVE,Moderate
Kiribati: Line Islands,21,33,46,ABOVE,High
Papua New Guinea,25,30,45,ABOVE,High
Nauru,29,33,38,AVG-ABOVE,Moderate
Vanuatu North,32,34,34,CLIMATOLOGY,Moderate-High
Palau,32,35,33,CLIMATOLOGY,Moderate
Tonga,32,36,32,CLIMATOLOGY,Moderate-High
Niue,34,34,32,CLIMATOLOGY,Moderate


### code the categories for the water stress calculations

In [173]:
dict_replace = {}

dict_replace['ABOVE'] = 4

dict_replace['AVG - ABOVE'] = 2
dict_replace['AVG-ABOVE'] = 2
dict_replace['AVG – ABOVE'] = 2
dict_replace['AVG–ABOVE'] = 2

dict_replace['NEAR NORMAL'] = 0
dict_replace['NEAR NORMAL '] = 0
dict_replace['CLIMATOLOGY'] = 0


dict_replace['AVG - BELOW'] = -3
dict_replace['AVG-BELOW'] = -3
dict_replace['AVG – BELOW'] = -3
dict_replace['AVG–BELOW'] = -3

dict_replace['BELOW'] = -5

dict_replace['CLIMATOLOGY'] = -666

In [174]:
forecast = forecast.replace({'CATEGORY':dict_replace})

In [175]:
forecast

,T1,T2,T3,country_fname,std,CONFIDENCE,CATEGORY
country_name,,,,,,,
Marshall Islands,3,17,80,Marshall_Islands,16.244933,Moderate-High,4
Federated States of Micronesia,20,22,58,Federated_States_of_Micronesia,20.239218,Moderate,4
Kiribati: Line Islands,21,33,46,Kiribati:_Line_Islands,11.440509,High,4
Papua New Guinea,25,30,45,Papua_New_Guinea,9.535794,High,4
Nauru,29,33,38,Nauru,28.115259,Moderate,2
Vanuatu North,32,34,34,Vanuatu_North,19.573168,Moderate-High,-666
Palau,32,35,33,Palau,30.241486,Moderate,-666
Tonga,32,36,32,Tonga,17.671547,Moderate-High,-666
Niue,34,34,32,Niue,21.046265,Moderate,-666


### changes the names of the columns etc and saves the forecast table for mapping 

In [176]:
table_for_mapping = forecast.copy()

In [177]:
table_for_mapping.loc[:,'COUNTRYNAM'] = table_for_mapping.index

In [178]:
table_for_mapping.rename({'CATEGORY':'Stress'}, axis=1, inplace=True)

In [179]:
table_for_mapping = table_for_mapping.loc[:,['Stress','COUNTRYNAM']]

In [180]:
table_for_mapping.index.name = None

In [181]:
table_for_mapping.to_csv(dpath / 'ICU_forecast_table_for_mapping.csv')

### Now read the last 6 months rainfall categories from TRMM

In [7]:
dpath

PosixPath('/home/nicolasf/operational/ICU/ops/bulletin/FMA_2020')

In [9]:
list(dpath.glob("*.csv"))

[PosixPath('/home/nicolasf/operational/ICU/ops/bulletin/FMA_2020/last_6_months_GPM_IMERG_categories.csv'),
 PosixPath('/home/nicolasf/operational/ICU/ops/bulletin/FMA_2020/last_6_months_TRMM_categories.csv')]

In [16]:
last_6_months = pd.read_csv(dpath / 'last_6_months_GPM_IMERG_categories.csv', index_col=0)

In [17]:
last_6_months.set_index('month', inplace=True)

In [15]:
last_6_months.columns = [x.replace('GPM_IMERG_','') for x in last_6_months.columns]

In [20]:
last_6_months.columns = last_6_months.columns.str.replace('GPM_IMERG_','')

In [21]:
last_6_months

,Papua_New_Guinea,Marshall_Islands,Kiribati_Phoenix_Islands,Northern_Cook_Islands,Samoa,Vanuatu_South,Guam,Solomon_Islands,Palau,Austral_Islands,...,Federated_States_of_Micronesia,Northern_Marianas,Wallis_and_Futuna,Tuvalu,Marquesas,Vanuatu_North,Nauru,Tonga,Southern_Cook_Islands,year
month,,,,,,,,,,,,,,,,,,,,,
8,Below,Below,Below,Well below,Well below,Below,Above,Well above,Normal,Below,...,Normal,Well above,Well above,Well above,Normal,Normal,Normal,Below,Below,2019
9,Well below,Above,Normal,Above,Below,Above,Above,Well above,Well below,Below,...,Below,Well above,Above,Well above,Normal,Below,Normal,Normal,Normal,2019
10,Above,Normal,Normal,Normal,Normal,Well below,Below,Below,Normal,Well below,...,Well below,Well above,Below,Below,Well below,Well below,Above,Above,Well above,2019
11,Above,Above,Below,Well below,Well below,Well below,Well above,Below,Normal,Well above,...,Above,Well above,Below,Well below,Well below,Well below,Normal,Below,Well above,2019
12,Well below,Below,Below,Normal,Well above,Well below,Well below,Well below,Below,Well below,...,Above,Well below,Normal,Well above,Well below,Well below,Above,Below,Well above,2019
1,Normal,Well above,Above,Below,Above,Below,Normal,Below,Below,Well below,...,Well below,Below,Above,Above,Above,Below,Above,Below,Below,2020


In [185]:
last_6_months = last_6_months.T

In [186]:
dict_replace = {}
dict_replace['Well below'] = -5 
dict_replace['Below'] = -3 
dict_replace['Normal'] = 0
dict_replace['Above'] = 2
dict_replace['Well above'] = 4

In [187]:
last_6_months.head()

month,7,8,9,10,11,12
Palau,Normal,Above,Well below,Below,Normal,Normal
Papua_New_Guinea,Below,Normal,Well below,Well above,Well above,Well below
Society_Islands,Well above,Below,Below,Below,Above,Well above
Niue,Above,Well above,Well above,Above,Below,Above
Kiribati_Phoenix_Islands,Well below,Normal,Above,Below,Below,Normal


In [188]:
last_6_months  = last_6_months.replace(dict_replace)

In [189]:
last_6_months = last_6_months.sort_index()

In [190]:
last_6_months.columns = range(-6,0)

In [191]:
last_6_months = last_6_months.rename(index=dict_countries)

In [192]:
last_6_months.head()

,-6,-5,-4,-3,-2,-1
American Samoa,4,-3,2,0,-5,4
Austral Islands,-3,-3,-5,0,4,-5
Federated States of Micronesia,0,4,-3,-5,4,4
Fiji,0,-3,2,4,0,2
Guam,-3,2,4,0,4,-5


In [193]:
last_6_months.index.name = 'country_name'

### now calculates the water stress itself 

In [194]:
water_stress = forecast.loc[:,['CATEGORY']]

In [195]:
water_stress = water_stress.merge(last_6_months, left_index=True, right_index=True)

### replace the code for climatology (-666) with 0 (i.e. equivalent to normal) for calculation of the water stress level

In [196]:
water_stress = water_stress.replace({'CATEGORY':{-666:0}})

In [197]:
water_stress

,CATEGORY,-6,-5,-4,-3,-2,-1
country_name,,,,,,,
Marshall Islands,4,-3,2,4,2,4,0
Federated States of Micronesia,4,0,4,-3,-5,4,4
Kiribati: Line Islands,4,4,0,0,-3,-3,-5
Papua New Guinea,4,-3,0,-5,4,4,-5
Nauru,2,2,0,0,2,2,4
Vanuatu North,0,0,4,2,-5,-3,-5
Palau,0,0,2,-5,-3,0,0
Tonga,0,4,-3,-3,2,-3,0
Niue,0,2,4,4,2,-3,2


In [198]:
water_stress

,CATEGORY,-6,-5,-4,-3,-2,-1
country_name,,,,,,,
Marshall Islands,4,-3,2,4,2,4,0
Federated States of Micronesia,4,0,4,-3,-5,4,4
Kiribati: Line Islands,4,4,0,0,-3,-3,-5
Papua New Guinea,4,-3,0,-5,4,4,-5
Nauru,2,2,0,0,2,2,4
Vanuatu North,0,0,4,2,-5,-3,-5
Palau,0,0,2,-5,-3,0,0
Tonga,0,4,-3,-3,2,-3,0
Niue,0,2,4,4,2,-3,2


In [199]:
water_stress.loc[:,'Stress'] = water_stress.loc[:,-1] +  water_stress.loc[:,-2] + \
0.8 * water_stress.loc[:,-3] + \
0.6 * water_stress.loc[:,-4] + \
0.4 * water_stress.loc[:,-5] + \
0.2 * water_stress.loc[:,-6] + \
water_stress.loc[:,'CATEGORY']

In [200]:
water_stress.head()

,CATEGORY,-6,-5,-4,-3,-2,-1,Stress
country_name,,,,,,,,
Marshall Islands,4,-3,2,4,2,4,0,12.2
Federated States of Micronesia,4,0,4,-3,-5,4,4,7.8
Kiribati: Line Islands,4,4,0,0,-3,-3,-5,-5.6
Papua New Guinea,4,-3,0,-5,4,4,-5,2.6
Nauru,2,2,0,0,2,2,4,10.0


### now determine the water stress catergory from the water stress value

In [201]:
bins = [-np.inf, -10, -7, -2, 2, 6, 11, np.inf]

In [202]:
labels = [-3, -2, -1, 0, 1, 2, 3]

In [203]:
water_stress.loc[:,'stress_category'] = pd.cut(water_stress.Stress.values.flatten(), bins=bins, labels=labels)

In [204]:
water_stress

,CATEGORY,-6,-5,-4,-3,-2,-1,Stress,stress_category
country_name,,,,,,,,,
Marshall Islands,4,-3,2,4,2,4,0,12.2,3
Federated States of Micronesia,4,0,4,-3,-5,4,4,7.8,2
Kiribati: Line Islands,4,4,0,0,-3,-3,-5,-5.6,-1
Papua New Guinea,4,-3,0,-5,4,4,-5,2.6,1
Nauru,2,2,0,0,2,2,4,10.0,2
Vanuatu North,0,0,4,2,-5,-3,-5,-9.2,-2
Palau,0,0,2,-5,-3,0,0,-4.6,-1
Tonga,0,4,-3,-3,2,-3,0,-3.6,-1
Niue,0,2,4,4,2,-3,2,5.0,1


In [205]:
water_stress.index.name = 'locationID'

In [206]:
dpath

PosixPath('/home/nicolasf/operational/ICU/ops/bulletin/JFM_2020')

### now saves for mapping 

In [207]:
water_stress.to_csv(dpath / 'ICU_stress_table_for_mapping.csv')

In [208]:
water_stress

,CATEGORY,-6,-5,-4,-3,-2,-1,Stress,stress_category
locationID,,,,,,,,,
Marshall Islands,4,-3,2,4,2,4,0,12.2,3
Federated States of Micronesia,4,0,4,-3,-5,4,4,7.8,2
Kiribati: Line Islands,4,4,0,0,-3,-3,-5,-5.6,-1
Papua New Guinea,4,-3,0,-5,4,4,-5,2.6,1
Nauru,2,2,0,0,2,2,4,10.0,2
Vanuatu North,0,0,4,2,-5,-3,-5,-9.2,-2
Palau,0,0,2,-5,-3,0,0,-4.6,-1
Tonga,0,4,-3,-3,2,-3,0,-3.6,-1
Niue,0,2,4,4,2,-3,2,5.0,1
